# *Sequence to Sequence Learning with Neural Networks* in PyTorch

Encoder-decoder sequence to sequence RNN implementation based on 2014 publication:

**Sequence to Sequence Learning with Neural Networks** - Ilya Sutskever, Oriol Vinyals, Quoc V. Le

https://arxiv.org/abs/1409.3215

## Dataset

In the project files(PyTorch dataset implementation fra_eng_dataset.py) I use small toy dataset(170K sentences) in fra-eng folder. But for this experiment I will try to use **WMT'14 English-German** dataset (4.5M sentences)

The dataset can be founde here:
https://nlp.stanford.edu/projects/nmt/

### PyTorch Dataset

In [1]:
from torch.utils.data import Dataset, DataLoader
import pickle
from nltk.tokenize import word_tokenize
import os
import torch
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import os


RNN_LAYERS = 4
RNN_HIDDEN_SIZE = 1024
IN_EMBEDDING_SIZE = 128
OUT_EMBEDDING_SIZE = 128
BATCH_SIZE = 50
EPOCHS = 50
MAXMAX_SENTENCE_LEN = 60


device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [2]:
def get_top_dictionary(text_corpus_path, top_n = 50000):

    print(f"Creating top dictionary from: {text_corpus_path}..")
    
    last_token_idx = 0
    token_dict = dict()
    token_counts_list = []
    token_idx_to_token = []
    
    with open(text_corpus_path, "r", encoding='utf-8') as f:
        
        for idx, line in enumerate(f.readlines()):
            
            if (idx+1)% 500000 == 0:
                print(f"Processed {idx+1} lines")

            line = line.replace('##AT##', '')
            token_list = word_tokenize(line)
            for token in token_list:
                token = token.lower()
                if token not in token_dict:
                    token_dict[token] = last_token_idx
                    token_idx_to_token.append(token)
                    token_counts_list.append((0,last_token_idx))
                    last_token_idx += 1

                token_idx = token_dict[token]
                count, _ = token_counts_list[token_idx]
                token_counts_list[token_idx] = (count+1,token_idx)
                
    token_counts_list = sorted(token_counts_list, reverse=True)
    
    top_token_list = []
    
    for idx, (count, token_idx) in enumerate(token_counts_list):
        top_token_list.append(token_idx_to_token[token_idx])
        
        if idx > top_n:
            break
    
    return top_token_list
        


class WMT14_en_de_Dataset(Dataset):
    def __init__(self, data_source_path = 'wmt14_en_de'):
        super().__init__()
        
        processed_data_path = "processed_data.pkl"
        top_en_tokens_path = "top_en_tokens.pkl"
        top_de_tokens_path = "top_de_tokens.pkl"
        
        self.sentence_list = []
        
        self.en_token_dict = dict()
        self.en_token_dict['<PAD>'] = 0
        self.en_token_dict['<EOS>'] = 1
        self.en_token_dict['<UNK>'] = 2
        self.en_last_token_idx = 2
        self.en_token_idx_to_text = ['<PAD>', '<EOS>', '<UNK>']
        
        self.de_token_dict = dict()
        self.de_token_dict['<PAD>'] = 0
        self.de_token_dict['<EOS>'] = 1
        self.de_token_dict['<UNK>'] = 2
        self.de_last_token_idx = 2
        self.de_token_idx_to_text = ['<PAD>', '<EOS>', '<UNK>']
        
        
        if os.path.exists(processed_data_path):
            with open(processed_data_path, 'rb') as f:
                pickle_data = pickle.load(f)
                self.sentence_list = pickle_data['sentence_list']
                self.en_last_token_idx = pickle_data['en_last_token_idx']
                self.de_last_token_idx = pickle_data['de_last_token_idx']
                self.en_token_idx_to_text = pickle_data['en_token_idx_to_text']
                self.de_token_idx_to_text = pickle_data['de_token_idx_to_text']
        else:
        
            en_sentences_path = os.path.join(data_source_path, "train.en")
            de_sentences_path = os.path.join(data_source_path, "train.de")
            
            if os.path.exists(top_en_tokens_path):
                with open(top_en_tokens_path, "rb") as f:
                    top_en_tokens = pickle.load(f)
            else:
                top_en_tokens = get_top_dictionary(en_sentences_path)
                with open(top_en_tokens_path, "wb") as f:
                    pickle.dump(top_en_tokens, f)
            
            
            for token in top_en_tokens:
                self.en_last_token_idx += 1
                self.en_token_dict[token] = self.en_last_token_idx
                self.en_token_idx_to_text.append(token)
                
 
            if os.path.exists(top_de_tokens_path):
                with open(top_de_tokens_path, "rb") as f:
                    top_de_tokens = pickle.load(f)
            else:
                top_de_tokens = get_top_dictionary(de_sentences_path)
                with open(top_de_tokens_path, "wb") as f:
                    pickle.dump(top_de_tokens, f)
            
            for token in top_de_tokens:
                self.de_last_token_idx += 1
                self.de_token_dict[token] = self.de_last_token_idx
                self.de_token_idx_to_text.append(token)         
                    
            
            with open(de_sentences_path, "r", encoding='utf-8') as de_f:
                with open(en_sentences_path, "r", encoding='utf-8') as en_f:
                    
                    print("Creating sentences from {de_sentences_path} and {en_sentences_path} coropuses")
                    
                    for idx, (de_sentence, en_sentence) in enumerate(zip(de_f.readlines(), en_f.readlines())):
                        
                        if (idx+1)%500000 == 0:
                            print(f"Processed {idx+1} lines")
                            
                        de_sentence = de_sentence.replace('##AT##', '')
                        en_sentence = en_sentence.replace('##AT##', '')
                        
                        en_token_sentence = []
                        de_token_sentence = []

                        en_token_list = word_tokenize(en_sentence)
                        for token in en_token_list:
                            token = token.lower()
                            if token in self.en_token_dict:
                                token_idx = self.en_token_dict[token]
                            else:
                                token_idx = self.en_token_dict['<UNK>']
                                
                            en_token_sentence.append(token_idx)

                        en_token_sentence.append(self.en_token_dict['<EOS>'])

                        de_token_list = word_tokenize(de_sentence)
                        for token in de_token_list:
                            token = token.lower()
                            if token in self.de_token_dict:
                                token_idx = self.de_token_dict[token]
                            else:
                                token_idx = self.de_token_dict['<UNK>']
                                    

                            de_token_sentence.append(token_idx)

                        de_token_sentence.append(self.de_token_dict['<EOS>'])

                        self.sentence_list.append(
                            dict(
                                en = en_token_sentence,
                                de = de_token_sentence
                            ))
                        
            with open(processed_data_path, "wb") as f:
                pickle_processed_data = dict(
                    sentence_list = self.sentence_list,
                    en_last_token_idx = self.en_last_token_idx,
                    de_last_token_idx = self.de_last_token_idx,
                    en_token_idx_to_text = self.en_token_idx_to_text,
                    de_token_idx_to_text = self.de_token_idx_to_text
                )
                pickle.dump(pickle_processed_data, f)
            
    def get_en_dict_size(self):
        return self.en_last_token_idx + 1
        
    def get_de_dict_size(self):
        return self.de_last_token_idx + 1
    
    def get_de_eos_code(self):
        return self.de_token_dict['<EOS>']
    
    def get_en_eos_code(self):
        return self.en_token_dict['<EOS>']

    def __len__(self):
        return len(self.sentence_list)

    def __getitem__(self, item):
        ret = dict()
        for key in self.sentence_list[item]:
            ret[key] = torch.tensor(self.sentence_list[item][key])
        return ret


def en_de_dataset_collate(data):

    en_sentences = []
    en_sentence_lens = []
    de_sentences = []
    de_sentence_lens = []
    
    en_sentences_sorted = []
    en_sentence_lens_sorted = []
    de_sentences_sorted = []
    de_sentence_lens_sorted = []
    
    for s in data:
        en_sentences.append(s['en'].unsqueeze(dim=1))
        en_sentence_lens.append(len(s['en']))
        de_sentences.append(s['de'].unsqueeze(dim=1))
        de_sentence_lens.append(len(s['de']))

    #Rearrange everything by de sentence lens
    sort_idxes = np.argsort(np.array(de_sentence_lens))[::-1]
    for idx in sort_idxes:
        en_sentences_sorted.append(en_sentences[idx])
        en_sentence_lens_sorted.append(en_sentence_lens[idx])
        de_sentences_sorted.append(de_sentences[idx])
        de_sentence_lens_sorted.append(de_sentence_lens[idx])
    
    return dict(
        en_sentences = en_sentences_sorted,
        en_lens = en_sentence_lens_sorted,
        de_sentences = de_sentences_sorted,
        de_lens = de_sentence_lens_sorted
    )

### Models

In [3]:
class RNN_encoder_model(nn.Module):
    def __init__(self, in_dict_size):
        super().__init__()
        
        self.in_dict_size = in_dict_size

        self.embedding = nn.Linear(
            in_dict_size, 
            IN_EMBEDDING_SIZE)
        
        
        self.hidden = None 
        self.cell = None
        
        self.rnn = nn.LSTM(
            input_size = IN_EMBEDDING_SIZE,
            hidden_size = RNN_HIDDEN_SIZE,
            num_layers = RNN_LAYERS
        )
        
    def init_hidden_and_cell(self):
        self.hidden = torch.randn(RNN_LAYERS, BATCH_SIZE, RNN_HIDDEN_SIZE).to(device)
        self.cell = torch.rand(RNN_LAYERS, BATCH_SIZE, RNN_HIDDEN_SIZE).to(device)
    
    def get_hidden_and_cell(self):
        return self.hidden, self.cell
    
    def forward(self, x):
        padded_sent_one_hot, sent_lens = x
        padded_sent_emb = self.embedding.forward(padded_sent_one_hot)
        packed = pack_padded_sequence(padded_sent_emb, sent_lens)
        packed, (self.hidden, self.cell) = self.rnn.forward(packed, (self.hidden,self.cell))
        padded, sent_lens = pad_packed_sequence(packed)

In [4]:
class RNN_decoder_model(nn.Module):
    def __init__(self, out_dict_size):
        super().__init__()
      
        self.in_embedding = nn.Linear(
            in_features=out_dict_size,
            out_features=IN_EMBEDDING_SIZE
        )

        self.rnn = nn.LSTM(
            input_size = IN_EMBEDDING_SIZE,
            hidden_size = RNN_HIDDEN_SIZE,
            num_layers = RNN_LAYERS
        )

        self.rnn_to_embedding = nn.Linear(
            in_features = RNN_HIDDEN_SIZE,
            out_features = OUT_EMBEDDING_SIZE
        )

        self.embedding_to_logit = nn.Linear(
            in_features = OUT_EMBEDDING_SIZE, 
            out_features = out_dict_size
        )

        self.softmax = nn.Softmax(dim=2)
    
    def init_hidden_and_cell(self, hidden, cell):
        self.hidden = hidden
        self.cell = cell
    
    
    def forward(self, out_eos_code, out_dict_size, max_sentence_len):
        batch_size = self.hidden.shape[1]
        prev_outp = (torch.ones(1, batch_size, 1) * out_eos_code).long()
        prev_outp = prev_outp.to(device)
        
        all_outp_prob = []
        
        for timestep in range(max_sentence_len):
            
            prev_outp_one_hot = torch.zeros(prev_outp.shape[0], prev_outp.shape[1], out_dict_size).to(device)
            prev_outp_one_hot = prev_outp_one_hot.scatter_(2,prev_outp.data,1)
            
            prev_outp_in_emb = self.in_embedding(prev_outp_one_hot)
         
            cur_outp_hid, (self.hidden, self.cell) = self.rnn.forward(prev_outp_in_emb, (self.hidden, self.cell))
            cur_outp_emb = self.rnn_to_embedding.forward(cur_outp_hid)
            cur_outp_logits = self.embedding_to_logit(cur_outp_emb)
            cur_outp_prob = self.softmax(cur_outp_logits)
            all_outp_prob.append(cur_outp_prob)
            
            prev_outp = torch.argmax(cur_outp_prob.data.to(device), dim=2, keepdim=True)
        
        all_outp_prob_tensor = torch.cat(all_outp_prob, dim=0)
    
        return all_outp_prob_tensor
   

In [ ]:
dataset = WMT14_en_de_Dataset()
sentences_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, collate_fn=en_de_dataset_collate)


In [ ]:
def print_results(in_sentence_list, out_sentence_list, pred_tensor):
    
    in_token_to_text = dataset.de_token_idx_to_text
    out_token_to_text = dataset.en_token_idx_to_text
    
    for s in range(len(in_sentence_list)):
        
        in_sent_text = []
        for in_token in in_sentence_list[s].squeeze():
            in_sent_text.append(in_token_to_text[in_token])
        
        print(f"\nGerman sentence is: {' '.join(in_sent_text)} ")
              
        out_sent_text = []
              
        for out_token in out_sentence_list[s].squeeze():
              out_sent_text.append(out_token_to_text[out_token])
        print(f"English sentence is: {' '.join(out_sent_text)}")
              
        pred_sent_text = []
        for ts in range(pred_tensor.shape[0]):
            pred_token = torch.argmax(pred_tensor[ts, s,:]).data
            pred_sent_text.append(out_token_to_text[pred_token])
              
            if pred_token == dataset.get_en_eos_code():
                break
        print(f"Translated English sentence is: {' '.join(pred_sent_text)}")


## Training

In [ ]:
rnn_encoder = RNN_encoder_model(dataset.get_de_dict_size()).to(device)
rnn_decoder = RNN_decoder_model(dataset.get_en_dict_size()).to(device)

trained_encoder_path = None
trained_decoder_path = None

trained_encoder_path = 'models/encoder_wmt14_de_en.pt'
trained_decoder_path = 'models/decoder_wmt14_de_en.pt'

if os.path.exists(trained_encoder_path):
    rnn_encoder.load_state_dict(torch.load(trained_encoder_path))
if os.path.exists(trained_decoder_path):
    rnn_decoder.load_state_dict(torch.load(trained_decoder_path))


params = list(rnn_encoder.parameters()) + list(rnn_decoder.parameters())
optimizer = torch.optim.Adam(params, lr = 1e-3)

steps = 0

for epoch in range(EPOCHS):
    
    print(f"Starting epoch {epoch} =====================")
    
    best_loss = 1e10
    loss_sum = 0
    
    for idx, sentences in enumerate(sentences_loader):

        rnn_encoder.init_hidden_and_cell()
       
   
        in_sentences = sentences['de_sentences']
        in_lens = sentences['de_lens']
        out_sentences = sentences['en_sentences']
        out_lens = sentences['en_lens']
        

        padded_in = pad_sequence(in_sentences, padding_value=0).to(device)
        padded_out = pad_sequence(out_sentences, padding_value=0).to(device)

        padded_in_one_hot = torch.zeros(padded_in.shape[0], padded_in.shape[1], dataset.get_de_dict_size()).to(device)
        padded_in_one_hot = padded_in_one_hot.scatter_(2,padded_in.data,1)
       
        rnn_encoder.forward((padded_in_one_hot, in_lens))
        hidden, cell = rnn_encoder.get_hidden_and_cell()
       
        rnn_decoder.init_hidden_and_cell(hidden,cell)
       
        max_sentence_len = padded_out.shape[0]
        if max_sentence_len > MAXMAX_SENTENCE_LEN:
            max_sentence_len = MAXMAX_SENTENCE_LEN
            
        y_pred = rnn_decoder.forward(dataset.get_en_eos_code(), dataset.get_en_dict_size(), max_sentence_len)
       
        steps += BATCH_SIZE
        if steps > 30000:
            print(f"Epoch {epoch} batch {idx}:")
            steps = 0
            print_results(in_sentences, out_sentences, y_pred.to('cpu').detach().data)
        
        padded_out = padded_out[0:max_sentence_len]
        padded_out_one_hot = torch.zeros(padded_out.shape[0], padded_out.shape[1], dataset.get_en_dict_size()).to(device)
        padded_out_one_hot = padded_out_one_hot.scatter_(2,padded_out.data,1)
       
        #Make all padded one-hot vectors to all zeros, which will make
        #padded components loss 0 and so they wont affect the loss
        padded_out_one_hot[:,:,0] = torch.zeros(max_sentence_len, padded_out_one_hot.shape[1])
        loss = torch.sum(-torch.log(y_pred + 1e-9) * padded_out_one_hot)
       
        loss_sum += loss.to('cpu').detach().data
       
        print(loss.to('cpu').detach().data)
       
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch} loss sum is {loss_sum}")
    if best_loss > loss_sum:
        best_loss = loss_sum
       
        models_path = "models"
        if not os.path.exists(models_path):
            os.mkdir(models_path)
       
        torch.save(rnn_encoder.state_dict(), trained_encoder_path)
        torch.save(rnn_decoder.state_dict(), trained_decoder_path)


Starting epoch 0 =====================
tensor(14275.0176)
tensor(14924.0752)
tensor(13532.8379)
tensor(13955.5273)
tensor(14642.2129)
tensor(11574.2012)
tensor(11795.3594)
tensor(11087.5947)
tensor(11644.8145)
tensor(10971.8691)
tensor(11413.5938)
tensor(10989.4414)
tensor(10158.7715)
tensor(10081.0586)
tensor(10347.5273)
tensor(10760.5117)
tensor(9626.1807)
tensor(9648.4668)
tensor(10180.6582)
tensor(9845.2930)
tensor(7993.2710)
tensor(9484.2217)
tensor(10539.1855)
tensor(9989.8496)
tensor(9184.6797)
tensor(9936.0645)
tensor(9800.8672)
tensor(9762.7969)
tensor(9268.4590)
tensor(9987.7549)
tensor(10592.6494)
tensor(9110.9551)
tensor(10491.8174)
tensor(8868.9229)
tensor(9384.7422)
tensor(8744.3789)
tensor(8604.2676)
tensor(11227.9834)
tensor(10370.6484)
tensor(8835.1270)
tensor(8785.3789)
tensor(7846.9824)
tensor(8940.2520)
tensor(8695.9414)
tensor(9104.2715)
tensor(9052.4365)
tensor(9247.2412)
tensor(9352.9727)
tensor(9253.6680)
tensor(10197.9189)
tensor(10286.1445)
tensor(8831.5537)
t

tensor(9160.4639)
tensor(9035.1475)
tensor(8908.4170)
tensor(8902.9648)
tensor(9235.2852)
tensor(8570.9102)
tensor(9268.4961)
tensor(9305.9121)
tensor(8922.1797)
tensor(7946.4795)
tensor(8876.3750)
tensor(8788.3184)
tensor(8449.8818)
tensor(8370.8975)
tensor(9334.4023)
tensor(9714.7207)
tensor(8571.1953)
tensor(9087.3438)
tensor(10192.9854)
tensor(9562.0918)
tensor(9582.8828)
tensor(8217.0928)
tensor(8959.6377)
tensor(9946.8828)
tensor(8573.3447)
tensor(9385.3721)
tensor(7856.9180)
tensor(9026.1748)
tensor(9246.5430)
tensor(9642.0332)
tensor(9447.1094)
tensor(8359.2383)
tensor(8200.0273)
tensor(9052.1660)
tensor(9858.7207)
tensor(10069.4004)
tensor(9525.8428)
tensor(8203.4033)
tensor(8680.0908)
tensor(9185.0469)
tensor(9029.3857)
tensor(8165.0137)
tensor(9486.8730)
tensor(9445.6924)
tensor(8419.4209)
tensor(8682.1426)
tensor(7902.6387)
tensor(8901.3389)
tensor(9554.5811)
tensor(9861.8242)
tensor(8995.1211)
tensor(8706.8086)
tensor(8483.6855)
tensor(8968.8604)
tensor(8911.5840)
tensor(8

tensor(9137.7168)
tensor(8889.5830)
tensor(8911.3223)
tensor(8335.3799)
tensor(9171.1152)
tensor(8881.8730)
tensor(9597.2383)
tensor(8714.9004)
tensor(9860.2617)
tensor(10097.4961)
tensor(9064.9238)
tensor(8503.0117)
tensor(10206.9297)
tensor(8819.5225)
tensor(9086.8730)
tensor(8468.2676)
tensor(9294.3340)
tensor(9655.8721)
tensor(8496.8867)
tensor(9434.7305)
tensor(8929.7324)
tensor(9494.9287)
tensor(7577.8398)
tensor(8857.1719)
tensor(9275.4365)
tensor(9130.1719)
tensor(8421.5908)
tensor(8613.9512)
tensor(9057.9521)
tensor(8620.8438)
tensor(8462.4238)
tensor(9171.7207)
tensor(8803.1660)
tensor(9443.9404)
tensor(9359.6631)
tensor(9436.0186)
tensor(10198.2031)
tensor(9618.3438)
tensor(8974.3564)
tensor(9074.5049)
tensor(9094.4570)
tensor(9833.6514)
tensor(9594.5088)
tensor(9518.2227)
tensor(8201.2012)
tensor(9237.3691)
tensor(9592.9863)
tensor(8104.4604)
tensor(9090.3916)
tensor(8909.4170)
tensor(8945.1348)
tensor(8612.5264)
tensor(7611.8423)
tensor(8258.2959)
tensor(9490.5820)
tensor(

tensor(9868.5713)
tensor(8135.7769)
tensor(9086.5586)
tensor(8515.8105)
tensor(8369.2256)
tensor(8351.2930)
tensor(8176.3164)
tensor(7968.0742)
tensor(9923.0088)
tensor(10182.9111)
tensor(9413.7539)
tensor(9242.3906)
tensor(9117.8086)
tensor(8455.1982)
tensor(10984.9453)
tensor(8977.0713)
tensor(8730.5352)
tensor(8609.2402)
tensor(9940.7314)
tensor(8376.1455)
tensor(8420.6855)
tensor(9429.7227)
tensor(9181.6875)
tensor(8986.4844)
tensor(7640.2734)
tensor(8622.0879)
tensor(9572.1914)
tensor(9930.1602)
tensor(7322.4043)
tensor(8296.9609)
tensor(10562.6562)
tensor(8140.4194)
tensor(9273.4414)
tensor(8490.7031)
tensor(9015.0430)
tensor(9185.2588)
tensor(9125.8125)
tensor(9186.2744)
tensor(9238.5830)
tensor(8742.2148)
tensor(8765.5352)
tensor(9275.2090)
tensor(9887.6953)
tensor(8528.0156)
tensor(9201.7324)
tensor(11111.4766)
tensor(10400.5352)
tensor(10781.4873)
tensor(9074.8516)
tensor(9051.0215)
tensor(8730.)
tensor(9699.0928)
tensor(9080.2363)
tensor(9318.8389)
tensor(8687.9629)
tensor(8

tensor(9868.2734)
tensor(11450.0459)
tensor(9878.4541)
tensor(9326.4238)
tensor(10104.3301)
tensor(9453.7168)
tensor(8779.9932)
tensor(9798.5859)
tensor(8113.4272)
tensor(8350.2314)
tensor(9419.9414)
tensor(8823.6553)
tensor(8908.9355)
tensor(8463.1416)
tensor(8105.8379)
tensor(10825.2158)
tensor(8725.2695)
tensor(8760.4727)
tensor(8927.0654)
tensor(8811.6592)
tensor(8564.3350)
tensor(7915.3354)
tensor(8961.9707)
tensor(9022.4082)
tensor(9973.4473)
tensor(9197.4473)
tensor(9110.4238)
tensor(8347.1621)
tensor(9314.9102)
tensor(8268.9570)
tensor(8670.8926)
tensor(8827.1494)
tensor(8883.5020)
tensor(10200.8750)
tensor(9581.6562)
tensor(9046.4873)
tensor(8965.7139)
tensor(10350.2715)
tensor(9941.1846)
tensor(8256.8896)
tensor(8091.1675)
tensor(9260.8809)
tensor(8798.6416)
tensor(9178.8672)
tensor(9207.0410)
tensor(9032.2148)
tensor(8734.9639)
tensor(8801.3096)
tensor(9314.1748)
tensor(8751.4668)
tensor(9400.7129)
tensor(8067.9111)
tensor(7944.8179)
tensor(8798.0742)
tensor(8766.1602)
tenso

...

tensor(8904.6719)
tensor(8461.6113)
tensor(8514.9512)
tensor(8258.0918)
tensor(10190.4297)
tensor(8609.2256)
tensor(10056.0664)
tensor(8726.1904)
tensor(8527.4893)
tensor(7946.8599)
tensor(7722.2622)
tensor(8067.6992)
tensor(7994.7710)
tensor(7990.1265)
tensor(8771.6387)
tensor(8775.3672)
tensor(8464.2910)
tensor(8227.3154)
tensor(10192.7480)
tensor(9326.3262)
tensor(8329.3633)
tensor(8073.6006)
tensor(8819.1914)
tensor(9654.9512)
tensor(10638.3682)
tensor(8951.3467)
tensor(8127.1514)
tensor(8938.2539)
tensor(8435.4941)
tensor(10380.9131)
tensor(8438.4482)
tensor(8760.4785)
tensor(8667.5723)
tensor(8411.4043)
tensor(9101.4316)
tensor(8593.1826)
tensor(8614.0996)
tensor(8260.4980)
tensor(8571.9277)
tensor(9271.6289)
tensor(9380.0059)
tensor(9235.2754)
tensor(8690.5400)
tensor(8723.9512)
tensor(9374.0742)
tensor(7878.7949)
tensor(8686.0586)
tensor(8095.7808)
tensor(9028.6953)
tensor(9097.1953)
tensor(8426.6279)
tensor(9419.5059)
tensor(10262.8750)
tensor(8072.8452)
tensor(10091.8438)
ten

tensor(8607.4590)
tensor(9710.5566)
tensor(8392.3477)
tensor(9183.5508)
tensor(8536.5088)
tensor(8616.2578)
tensor(7580.6348)
tensor(8257.6328)
tensor(8861.2031)
tensor(8627.8848)
tensor(8851.6426)
tensor(8212.4180)
tensor(9583.9795)
tensor(9624.7383)
tensor(7926.2412)
tensor(9172.4922)
tensor(8771.4404)
tensor(9384.8516)
tensor(9283.9199)
tensor(8187.2842)
tensor(8205.9443)
tensor(9353.8867)
tensor(8972.9717)
tensor(10485.6553)
tensor(8424.0869)
tensor(7354.1270)
tensor(8935.3770)
tensor(9264.6113)
tensor(8891.5117)
tensor(8443.2305)
tensor(9767.9512)
tensor(10103.9541)
tensor(8048.4224)
tensor(9215.3203)
tensor(9106.8945)
tensor(9021.4795)
tensor(8763.2598)
tensor(9032.7510)
tensor(9522.0645)
tensor(8614.1328)
tensor(9178.4121)
tensor(8207.1035)
tensor(7918.1040)
tensor(8499.6094)
tensor(8942.8760)
tensor(9226.2764)
tensor(8616.6807)
tensor(9612.4150)
tensor(10705.3867)
tensor(9235.5098)
tensor(9527.8936)
tensor(8218.0898)
tensor(8877.5137)
tensor(9561.6035)
tensor(8554.6035)
tensor(

tensor(8932.5020)
tensor(9636.1904)
tensor(9274.4277)
tensor(8061.0684)
tensor(9216.5361)
tensor(8375.2080)
tensor(8927.4072)
tensor(9039.3242)
tensor(8241.0615)
tensor(10229.2812)
tensor(8514.4336)
tensor(7536.0874)
tensor(8502.6025)
tensor(7747.9697)
tensor(8347.9082)
tensor(7859.1924)
tensor(8760.4648)
tensor(9236.5869)
tensor(8347.8457)
tensor(8901.7725)
tensor(8392.8223)
tensor(9324.1777)
tensor(7994.0713)
tensor(8443.2520)
tensor(8466.0596)
tensor(10178.6475)
tensor(7897.7134)
tensor(8171.8911)
tensor(8215.9062)
tensor(8743.0361)
tensor(8616.5098)
tensor(8077.8364)
tensor(7934.7827)
tensor(8913.5137)
tensor(8371.3574)
tensor(8859.2842)
tensor(8367.2080)
tensor(8496.5137)
tensor(8322.2637)
tensor(9136.3086)
tensor(9237.8633)
tensor(9691.4473)
tensor(8411.4033)
tensor(8889.2734)
tensor(9889.5635)
tensor(9008.9844)
tensor(8877.9717)
tensor(9273.8555)
tensor(7051.1055)
tensor(8270.6016)
tensor(8538.5557)
tensor(8333.9375)
tensor(8253.0020)
tensor(8989.2656)
tensor(8659.7441)
tensor(9

tensor(9300.7314)
tensor(9417.6562)
tensor(8913.7422)
tensor(8102.1699)
tensor(8845.0186)
tensor(8172.5068)
tensor(9215.9453)
tensor(9096.6084)
tensor(9400.0391)
tensor(9072.6660)
tensor(8168.3535)
tensor(7970.2729)
tensor(7809.2876)
tensor(8107.0918)
tensor(8020.4834)
tensor(8501.4717)
tensor(6840.8691)
tensor(8962.1143)
tensor(9054.6191)
tensor(6576.2480)
tensor(9627.3418)
tensor(8374.3086)
tensor(8218.5039)
tensor(9784.6641)
tensor(8466.6055)
tensor(8269.5977)
tensor(8561.1191)
tensor(8476.1855)
tensor(10151.0352)
tensor(8226.3408)
tensor(7846.9121)
tensor(8708.5146)
tensor(9326.3359)
tensor(8228.6641)
tensor(9084.7891)
tensor(8243.8076)
tensor(9239.3955)
tensor(9123.4219)
tensor(8527.6016)
tensor(8180.8208)
tensor(8896.3984)
tensor(8851.8730)
tensor(8187.8677)
tensor(8420.4336)
tensor(8985.3555)
tensor(9385.5293)
tensor(8710.8613)
tensor(8358.4336)
tensor(9429.4688)
tensor(8459.0234)
tensor(8954.1113)
tensor(9771.6631)
tensor(8815.8027)
tensor(9525.4551)
tensor(7950.3647)
tensor(89

tensor(8148.2305)
tensor(7932.1411)
tensor(8638.2725)
tensor(7805.8975)
tensor(8209.9316)
tensor(9609.9297)
tensor(8252.3252)
tensor(8594.6191)
tensor(10143.6865)
tensor(8337.1387)
tensor(8494.9531)
tensor(9448.0264)
tensor(7882.2510)
tensor(8236.0830)
tensor(8774.8730)
tensor(9348.2168)
tensor(7891.7012)
tensor(8474.9131)
tensor(9092.6660)
tensor(8467.1475)
tensor(9038.4541)
tensor(8533.9629)
tensor(9228.0977)
tensor(9450.3770)
tensor(8253.0967)
tensor(9069.4766)
tensor(8371.9043)
tensor(7556.6436)
tensor(9145.6172)
tensor(7929.2002)
tensor(8827.0557)
tensor(8794.8779)
tensor(8978.8477)
tensor(8035.7861)
tensor(7615.1338)
tensor(8568.5762)
tensor(9084.4170)
tensor(7815.0479)
tensor(8449.0850)
tensor(8023.4731)
tensor(8459.5742)
tensor(9491.7705)
tensor(9617.1367)
tensor(8362.1797)
tensor(8157.8281)
tensor(7857.8330)
tensor(7865.8237)
tensor(9981.8887)
tensor(8371.1787)
tensor(8908.2773)
tensor(8915.9023)
tensor(8544.3506)
tensor(8160.6543)
tensor(7991.3760)
tensor(7903.7305)
tensor(81